Read blocks information from block.csv

In [8]:
import csv
import numpy as np

block_csv = csv.reader(open("block.csv"))
block_data = np.array(list(block_csv))
block_data = (block_data[1:,:4]).astype(np.int32)

In [9]:
block_data[0,:]

array([  135, 44739,   201,   305], dtype=int32)

Set the geotransform

In [13]:
left = np.min(block_data[:, 2])*1000
top = (np.max(block_data[:, 3]) + 1)*1000
tif_geotrans = [left, 1000, 0, top, 0, -1000]

In [14]:
tif_geotrans

[9000, 1000, 0, 630000, 0, -1000]

Set the projection

In [12]:
from osgeo import gdal
from osgeo import osr

srs = osr.SpatialReference()            # establish encoding
srs.ImportFromEPSG(28992)               # NL projection

0

In [25]:
row_num = np.max(block_data[:, 3]) - np.min(block_data[:, 3]) + 1
lat_max = np.max(block_data[:, 3])
col_num = np.max(block_data[:, 2]) - np.min(block_data[:, 2]) + 1
lon_min = np.min(block_data[:, 2])

In [26]:
row_num, col_num, lat_max, lon_min

(325, 269, 629, 9)

In [27]:
block_raster = np.zeros((row_num, col_num))
for block_i in range(len(block_data)):
    block_id = block_data[block_i, 1]
    block_row = lat_max - block_data[block_i, 3]
    block_col = block_data[block_i, 2] - lon_min
    block_raster[block_row, block_col] = block_id

In [28]:
block_raster.shape

(325, 269)

Save tiff file.

In [31]:
block_raster_filename = '0_data/block_raster.tif'
# create the 3-band raster file
blk_ds = gdal.GetDriverByName('GTiff').Create(block_raster_filename, int(col_num), int(row_num), 1, gdal.GDT_Int32)

blk_ds.SetGeoTransform(tif_geotrans)    # specify coords
blk_ds.SetProjection(srs.ExportToWkt()) # export coords to file
blk_ds.GetRasterBand(1).WriteArray(block_raster)   # write data to the raster
blk_ds.FlushCache()                     # write to disk
blk_ds = None